In [ ]:
# importing some lib...:p  KINDLY SEE ANOTHER FILE IN REPO. PLZ LOOK COMMENTs

import tensorflow as tf 
from tensorflow.keras.layers import  Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from collections import deque
import numpy as np
import gym
import random
import os
from tensorflow.keras.losses import Huber
import tqdm
from skimage import transform 
from skimage.color import rgb2gray

In [ ]:
# testing gpu availability and tensorflow version
print("GPU Available: ", tf.test.is_gpu_available())
tf.__version__

In [ ]:
! pip install gym[atari]

In [ ]:
# loading environment
env = gym.make('Breakout-v0')

In [ ]:
# getting idea about environment
print("The size of our frame is: ", env.observation_space)
print("The action size is : ", env.action_space.n)

In [ ]:
# getting different actions
possible_actions = np.array(np.identity(env.action_space.n,dtype=int).tolist())
print(possible_actions)

In [ ]:

# processing images from environment

def image_process(frame):
    
    gray = rgb2gray(frame)
    cropped_frame = gray[8:-12,4:-12]
    normalized_frame = cropped_frame/255.0
    preprocessed_frame = transform.resize(normalized_frame, [105,80])
    return preprocessed_frame 
    

In [ ]:
# 4 images represents a single state


stack_size = 4 


stacked_frames  =  deque([np.zeros((105,80), dtype=np.int) for i in range(stack_size)], maxlen=4)


def stack_state(stacked_frames, state, is_new_episode):
    
    frame = image_process(state)

    if is_new_episode:

        stacked_frames = deque([np.zeros((105,80), dtype=np.int) for i in range(stack_size)], maxlen=4)


        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)


        stacked_state = np.stack(stacked_frames, axis=2)

    else:

        stacked_frames.append(frame)


        stacked_state = np.stack(stacked_frames, axis=2) 
    stac =  np.array(stacked_state).reshape(-1, 105, 80, 1)      
   # print(stac.shape)
    return stac, stacked_frames
    

In [ ]:
#our agent 007 :P

class my_agent:
    def __init__(self,env):
        self.env = env
        self.state_input = Input((105,80,1), name='state_input')
        #self.action_input = action
        self.gamma = 0.6
        self.epsilon = 0.1
        self.epsilon_decay = 0.00001 
        self.memory = deque(maxlen=50000)
        self.q_net = self.main_model()
        self.target_net = self.main_model()
        self.alighn_target_model()
        self.epsilon_min = 0.01
        
        
    def main_model(self):
        x = Conv2D(16,8,(4,4), activation='relu')(self.state_input)
        x = Conv2D(32,4,(2,2), activation = 'relu')(x)
        x = Flatten()(x)
        x = Dense(256)(x)
        y = Dense(env.action_space.n)(x)
        #q_val = tf.reduce_sum(tf.multiply(y, self.actions_input))
        model = Model(inputs=[self.state_input], outputs= y)
        optimizer = Adam(learning_rate=0.01)
        huber = Huber()
        model.compile(optimizer, loss=huber)
        return model
        
        
        
    def store(self, state, action, reward, next_state, terminated):
        self.memory.append((state, action, reward, next_state, terminated))
        
    
    def _update_epsilon(self):
        self.epsilon -= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        
        
   
    def alighn_target_model(self):
        self.target_net.set_weights(self.q_net.get_weights())     
        
     
    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            self._update_epsilon()
            return self.env.action_space.sample()
        
        state= state        
        q_values = self.q_net.predict(state)
        a = np.argmax(q_values[0])
        #print(a)
        return np.array([a])
        
   
    def retrain(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        for state, action, reward, next_state, terminated in minibatch:
            
            state = state  
      
            next_state = next_state
           
            
            target = self.q_net.predict(state)
            #print(target)
            if terminated:
                target[0][action] = reward
            else:
                t = self.target_net.predict(next_state)
                target[0][action] = reward + self.gamma * np.amax(t)
            
            self.q_net.fit(state, target, epochs=1, verbose=0)

 
     
    def load(self, path):
        self.q_net.load_weights(path)

    def save(self, path):
        self.q_net.save_weights(path)
        

In [ ]:
state = env.reset()

agent = my_agent(env)

# some hyperparameter...

batch_size = 64                             
num_of_episodes = 1000
timesteps_per_episode = 1000
agent.q_net.summary()

In [ ]:
# our agent will play and learn


for e in tqdm.tqdm(range(0, num_of_episodes)):
    
    state = env.reset()
    state , stacked_frames = stack_state(stacked_frames, state, True)

    reward = 0
    terminated = False

    for timestep in range(timesteps_per_episode):
        env.render()
        state = state 
        
        action = agent.act(state)
        #action = np.array([1])
        print(action)
        
        next_state, reward, terminated, info = env.step(action)
        next_state , stacked_frames = stack_state(stacked_frames, next_state , False)

        agent.store(state, action, reward, next_state, terminated)

        state  = next_state


        if terminated:
         
            print("Total reward is {}".format(rewards))
            agent.alighn_target_model()
            break

        if len(agent.memory) > batch_size:
            agent.retrain(batch_size)